In [18]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Connect to the DB

In [19]:
#time now
from datetime import datetime
start = datetime.now()

params = {

    'Trusted_Connection': 'Yes',
    'Driver': '{ODBC Driver 17 for SQL Server}',
    'Server': 'kosmos',
    'Database': 'SATSA_ARKIV'
}

# Create a connection string
conn_str = ';'.join([f"{k}={v}" for k, v in params.items()])

try:
    # Create a SQLAlchemy engine
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")
    
    print("Connection successful")
    
    # Define table names
    ipt1_table = "archive.ipt1_20130909"
    mortality_table = "archive.resp_0121_mortality2022"

    # Query to join ipt1_table and mortality_table
    query = f"""
    SELECT i.*, m.birthdate1, m.death_yrmon, m.age_death
    FROM {ipt1_table} i
    JOIN {mortality_table} m
    ON i.TWINNR = m.TWINNR
    """

    combined_tables = pd.read_sql_query(query, engine)
   
    #write to file
    # combined_tables.to_csv('combined_tables.csv', index=False)

    print(f"Query executed successfully. DataFrame shape: {combined_tables.shape}")

   
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the connection
    if 'engine' in locals():
        engine.dispose()

Connection successful
Query executed successfully. DataFrame shape: (645, 424)


Find average death year if needed

In [22]:
#find average death year. Use the first four digits of death_yrmon. Skip rows where death_yrmon is missing.
# death_years = combined_tables['death_yrmon'].str[:4].astype(float)
# avg_death_year = death_years.mean()
# print(f"\nAverage death year: {avg_death_year}")

Add labels

In [20]:
#Add row to combined_tables called 'labels' with values '0' or '1' based on if death_yrmon begins with a number greater than 1996. If no death_yrmon is present, set the value to None.
combined_tables['labels'] = combined_tables['death_yrmon'].apply(lambda x: '1' if x and x[:4] > '2005' 
                                                                 else '0' if x and x[:4] <= '2005' 
                                                                 else None)

Train a Random Forest model to predict the 'labels' column. twinnr is the unique identifier for each row.

Preprocessing

In [21]:
# Drop rows with missing values in the 'labels' column
combined_tables = combined_tables.dropna(subset=['labels'])

# Define the features and target
X = combined_tables.drop(['labels', 'twinnr', 'death_yrmon','birthdate1', 'age_death'], axis=1)
y = combined_tables['labels']

# Identify non-numeric columns
non_numeric_cols = X.select_dtypes(include=['object']).columns
print(f"Non-numeric columns: {non_numeric_cols}")

# Convert non-numeric columns to numeric using one-hot encoding
X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)

print(X.head())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a more comprehensive parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'class_weight': [None, 'balanced'],
    'ccp_alpha': [0.0, 0.1]
}

Non-numeric columns: Index(['punching'], dtype='object')
   weight  length    hip  waist  navel  widepart  bpsres  bpdres  rpulse  \
0    55.0   148.0   99.0   89.0   98.0      97.0   165.0    78.0    60.0   
1    63.0   152.0  100.0   81.0  100.0     103.0   150.0    72.0    84.0   
2    50.0   162.0   84.0   79.0    NaN       NaN   164.0    85.0    83.0   
3     NaN     NaN    NaN    NaN    NaN       NaN     NaN     NaN     NaN   
4    60.0   172.0   88.0   82.0   91.0      93.0   162.0    80.0    56.0   

   bpssta  ...  wrist  adltot  adltime  bestvis  strength     fine_1  \
0   170.0  ...    0.0     0.0      NaN      0.0       0.0  17.333333   
1   140.0  ...    0.0     5.0      8.0      0.0      10.0   8.000000   
2   100.0  ...    0.0     4.0      NaN      0.0      17.0   9.333333   
3   150.0  ...    1.0     0.0      NaN      0.0       8.0  16.000000   
4   160.0  ...    0.0     5.0     13.0      0.0      14.0   8.000000   

      bals_1  flex_1    motor_1  punching_easyp  
0  

Model training

In [23]:

# Create base model
rf = RandomForestClassifier(random_state=42)

#Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=1, scoring='accuracy')

# Fit GridSearchCV
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 2592 candidates, totalling 7776 fits


KeyboardInterrupt: 

Model evaluation

In [ ]:
# Get best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {best_score:.4f}")

# Get best model
best_rf = grid_search.best_estimator_

# Make predictions on test set
y_pred = best_rf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest set accuracy: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = best_rf.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': X.columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(10))
plt.title('Top 10 Most Important Features')
plt.tight_layout()
plt.show()

# Learning curves
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    best_rf, X, y, cv=5, n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 10), scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training score')
plt.plot(train_sizes, test_mean, label='Cross-validation score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Number of training examples')
plt.ylabel('Accuracy score')
plt.title('Learning Curve for Random Forest')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

print("total time ", datetime.now() - start)

NameError: name 'grid_search' is not defined